In [ ]:
import boto3
import requests
import datetime
import os
import pandas as pd

s3 = boto3.client("s3")

# Diccionario geo_id -> comunidad
geo_dict = {
    4:"Andalucía", 5:"Aragón", 6:"Cantabria", 7:"Castilla la Mancha",
    8:"Castilla y León", 9:"Cataluña", 10:"País Vasco",
    11:"Principado de Asturias", 8744:"Comunidad de Ceuta",
    8745:"Comunidad de Melilla", 13:"Comunidad de Madrid",
    14:"Comunidad de Navarra", 15:"Comunidad Valenciana",
    16:"Extremadura", 17:"Galicia", 8743:"Islas Baleares",
    8742:"Islas Canarias", 20:"La Rioja", 21:"Región de Murcia"
}

lista_de_geo_ids = list(geo_dict.keys())
url = "https://apidatos.ree.es/es/datos/demanda/evolucion"

def lambda_handler(event, context):
    hoy = datetime.date.today()
    start_date = hoy.strftime("%Y-%m-%dT00:00")
    end_date = hoy.strftime("%Y-%m-%dT23:59")

    registros = []

    # === Descargar datos de la API ===
    for geo_id in lista_de_geo_ids:
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": "day",
            "geo_limit": "ccaa",
            "geo_ids": geo_id
        }
        resp = requests.get(url, params=params, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        for v in data["included"][0]["attributes"]["values"]:
            registros.append({
                "datetime": v["datetime"],
                "value": v["value"],
                "geo_id": geo_id,
                "comunidad": geo_dict[geo_id]
            })

    df = pd.DataFrame(registros)

    # === Guardar un CSV por cada CCAA en /tmp ===
    bucket_name = os.environ["BUCKET_NAME"]   # nombre del bucket como variable de entorno
    prefix = f"demanda_diaria/{hoy}/"         # carpeta en S3

    for ccaa in df["comunidad"].unique():
        df_filtrado = df[df["comunidad"] == ccaa].copy()
        nombre = ccaa.replace(" ", "_") + ".csv"
        ruta_local = f"/tmp/{nombre}"
        ruta_s3 = prefix + nombre

        # Guardar localmente
        df_filtrado.to_csv(ruta_local, index=False, encoding="utf-8")

        # Subir a S3
        s3.upload_file(ruta_local, bucket_name, ruta_s3)
        print(f"Subido: s3://{bucket_name}/{ruta_s3}")

    return {
        "statusCode": 200,
        "body": f"Archivos diarios guardados en s3://{bucket_name}/{prefix}"
    }